# Scratchbook

## Imports

In [47]:
from torchvision import models
from torchvision import transforms
import PIL.Image as Image
import numpy as np

## Model

In [48]:
# Instanciate the model
model = models.vgg19()

In [49]:
# Create an empty list
features_blobs = list()

# Create hook to dump features maps into the list created above
def hook_feature(module, input, output): 
    features_blobs.append(output.data.cpu().numpy())

In [51]:
# Get model features
features = model._modules["features"]
features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace)
  (18): MaxPool2d(kernel_size=2, stride=2, padding=0, 

In [53]:
# We can hook any layer from above
features._modules.get('0').register_forward_hook(hook_feature);

## Data

In [31]:
img = Image.open("./examples/pebbles.jpg")

In [55]:
# Preprocess the image so that it can be fed to the network
preprocess = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   #normalize
])

In [56]:
img_tensor = preprocess(img)
# ``unsqueeze_`` means the transformation is made inplace
# Here unsqueeze enables us to create a batch of 1 image
img_tensor.unsqueeze_(0)

tensor([[[[0.7059, 0.7529, 0.7765,  ..., 0.8118, 0.8353, 0.8314],
          [0.6863, 0.7647, 0.7961,  ..., 0.7765, 0.7882, 0.7686],
          [0.7137, 0.7686, 0.7725,  ..., 0.7882, 0.7765, 0.7529],
          ...,
          [0.7176, 0.7255, 0.6863,  ..., 0.1373, 0.1451, 0.1529],
          [0.5882, 0.6157, 0.6000,  ..., 0.1333, 0.1373, 0.1333],
          [0.4627, 0.5137, 0.5255,  ..., 0.1333, 0.1255, 0.1176]],

         [[0.7216, 0.7412, 0.7373,  ..., 0.8353, 0.8510, 0.8471],
          [0.7020, 0.7529, 0.7569,  ..., 0.7922, 0.7843, 0.7647],
          [0.7255, 0.7451, 0.7333,  ..., 0.7882, 0.7569, 0.7294],
          ...,
          [0.6431, 0.6510, 0.6314,  ..., 0.1529, 0.1686, 0.1765],
          [0.5098, 0.5412, 0.5451,  ..., 0.1725, 0.1765, 0.1765],
          [0.3765, 0.4392, 0.4706,  ..., 0.1765, 0.1765, 0.1686]],

         [[0.7647, 0.7843, 0.7804,  ..., 0.8353, 0.8627, 0.8588],
          [0.7373, 0.7961, 0.7922,  ..., 0.7961, 0.8039, 0.7843],
          [0.7529, 0.7843, 0.7686,  ..., 0

In [57]:
# Capture the features outputs at the layer given above
model.forward(img_tensor)
array = np.array(features_blobs[0])
array.shape

(1, 64, 224, 224)

## Refactoring

In [63]:
def get_output_from_layer(model, base_img_path, layer_id):
    """
    Get the features maps outputs from any layer
    using a forward pass from a base image
    in a VGG model
    
    Parameters
    ------------
    - model : torchvision.models.vgg.VGG
        Model to be investigated
        
    - base_img_path : string
        Path of the image to be used as input
        
    - layer_id : int
        Layer Number.
        Validity range: 0 - 36 (included)
        See ``model._modules['features']``
        for more details
        
    Returns
    ------------
    - layer_output : numpy.ndarray
        Array of features maps activations
    """
    if not layer_id in np.arange(0, 37):
        raise ValueError("``layer_id`` argument invalid.  "
                         f"Got: {layer_id}.  "
                         "Expected a value between 0 and 36 included.  ")
    
    
    # Create an empty list
    features_blobs = list()

    # Create hook to dump features maps into the list created above
    def hook_feature(module, input, output): 
        features_blobs.append(output.data.cpu().numpy())
        
    # Get model features
    features = model._modules["features"]
    # We can hook any layer from above
    features._modules.get(str(layer_id)).register_forward_hook(hook_feature);
    
    # Load image
    img = Image.open(base_img_path)
    
    # Preprocess the image so that it can be fed to the network
    preprocess = transforms.Compose([
       transforms.Resize(256),
       transforms.CenterCrop(224),
       transforms.ToTensor(),
       #normalize
    ])
    img_tensor = preprocess(img)
    img_tensor = img_tensor.unsqueeze(0)
    
    # Capture the features outputs at the layer given above
    model.forward(img_tensor)
    layer_output = np.array(features_blobs[0])
    
    return layer_output

In [64]:
layer_output = get_output_from_layer(model=model, base_img_path="./examples/pebbles.jpg", layer_id=5)
layer_output

array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.00540819],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.00177113],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],

        [[0.03042456, 0.17518562, 0.10294002, ..., 0.12878014,
          0.14633478, 0.11153839],
         [0.        , 0.17331523, 0.09561723, ..., 0.10231358,
          0.13772708, 0.20038472],
         [0.        , 0.14968084, 0.07567233, ..., 0.08151257,
          0.1288407 , 0.19302253],
         ...,
         [0.        , 0.13904926, 0.09470867, ..., 0.01174694,
          0.01564335, 0.0